In [1]:
from caffe import layers as L, params as P
import caffe

In [2]:
caffe.set_mode_gpu()
caffe.set_device(1)

In [ ]:
def net(data_set, batch_size, scope):
    n = caffe.NetSpec()
    n.data, n.label = L.ImageData(batch_size=batch_size, source=data_set, is_color=False, transform_param=dict(crop_size=100), ntop=2)
    
    n.conv1_1 = L.Convolution(n.data, kernel_size=3, pad=1, num_output=64, weight_filler=dict(type='xavier'))
    n.batch1_1 = L.BatchNorm(n.conv1_1)
    n.relu1_1 = L.ReLU(n.batch1_1, in_place=True)
    n.conv1_2 = L.Convolution(n.relu1_1, kernel_size=3, pad=1, num_output=64, weight_filler=dict(type='xavier'))
    n.batch1_2 = L.BatchNorm(n.conv1_2)
    n.relu1_2 = L.ReLU(n.batch1_2, in_place=True)
    n.pool1 = L.Pooling(n.relu1_2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv2_1 = L.Convolution(n.pool1, kernel_size=3, pad=1, num_output=128, weight_filler=dict(type='xavier'))
    n.batch2_1 = L.BatchNorm(n.conv2_1)
    n.relu2_1 = L.ReLU(n.batch2_1, in_place=True)
    n.conv2_2 = L.Convolution(n.batch2_1, kernel_size=3, pad=1, num_output=128, weight_filler=dict(type='xavier'))
    n.batch2_2 = L.BatchNorm(n.conv2_2)
    n.relu2_2 = L.ReLU(n.batch2_2, in_place=True)
    n.pool2 = L.Pooling(n.relu2_2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv3_1 = L.Convolution(n.pool2, kernel_size=3, pad=1, num_output=96, weight_filler=dict(type='xavier'))
    n.batch3_1 = L.BatchNorm(n.conv3_1)
    n.relu3_1 = L.ReLU(n.batch3_1, in_place=True)
    n.conv3_2 = L.Convolution(n.relu3_1, kernel_size=3, pad=1, num_output=96, weight_filler=dict(type='xavier'))
    n.batch3_2 = L.BatchNorm(n.conv3_2)
    n.relu3_2 = L.ReLU(n.batch3_2, in_place=True)
    n.conv3_3 = L.Convolution(n.relu3_2, kernel_size=3, pad=1, num_output=96, weight_filler=dict(type='xavier'))
    n.batch3_3 = L.BatchNorm(n.conv3_3)
    n.relu3_3 = L.ReLU(n.batch3_3, in_place=True)
    n.pool3 = L.Pooling(n.relu3_3, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv4_1 = L.Convolution(n.pool3, kernel_size=3, num_output=128, weight_filler=dict(type='xavier'))
    n.batch4_1 = L.BatchNorm(n.conv4_1)
    n.relu4_1 = L.ReLU(n.batch4_1, in_place=True)
    n.conv4_2 = L.Convolution(n.relu4_1, kernel_size=3, num_output=128, weight_filler=dict(type='xavier'))
    n.batch4_2 = L.BatchNorm(n.conv4_2)
    n.relu4_2 = L.ReLU(n.batch4_2, in_place=True)
    n.pool4 = L.Pooling(n.relu4_2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.fc1 =   L.InnerProduct(n.pool4, num_output=1536, weight_filler=dict(type='xavier'))
    n.relu_a1 = L.ReLU(n.fc1, in_place=True)
    n.fc2 = L.InnerProduct(n.relu_a1, num_output=600, weight_filler=dict(type='xavier'))
    n.relu_a2 = L.ReLU(n.fc2, in_place=True)
    n.fc3 = L.InnerProduct(n.relu_a2, num_output=300, weight_filler=dict(type='xavier'))
    n.relu_a3 = L.ReLU(n.fc3, in_place=True)
    
    n.score = L.InnerProduct(n.relu_a3, num_output=11, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.score, n.label)
    
    n.accuracy_top1 = L.Accuracy(n.score, n.label)
        
    return n.to_proto()
    
with open('proto/vgg10_train.prototxt', 'w') as f:
    f.write(str(net('resources/train.txt', 64, 'TRAIN')))

with open('proto/vgg10_test.prototxt', 'w') as f:
    f.write(str(net('resources/test.txt', 64, 'TEST')))